In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

In [20]:
def load_csv(datapath, dirname, filename):
    data_dirpath = os.path.join(datapath, dirname)
    if os.path.isdir(data_dirpath):
        data_filepath = os.path.join(data_dirpath, filename)
        
        if os.path.isfile(data_filepath):
            data = pd.read_csv(data_filepath)
            print("Loaded %s Data" %(dirname))
            
            return data
        
        else:
            print("Couldn't find file: %s" %(data_filepath))
            return None
    
    else:
        print("Couldn't find dir: %s" %(data_dirpath))
        return None

def load_data(datapath):
    """
    Function to load all csv files
    Input:  Path to Main Data dir
    Output: Pandas dataframes for each csv
    """
    
    nrc_text_data = load_csv(datapath, 'Text', 'nrc.csv')
    liwc_text_data = load_csv(datapath, 'Text', 'liwc.csv')
    relation_data = load_csv(datapath, 'Relation', 'Relation.csv')
    profile_data = load_csv(datapath, 'Profile', 'Profile.csv')
    image_data = load_csv(datapath, 'Image', 'oxford_merged.csv')
            
    return nrc_text_data, liwc_text_data, relation_data, profile_data, image_data

In [53]:
data = load_data('C:\\Users\\jonat\\Documents\\School\\DataScience\\Project\\Train')

Loaded Text Data
Loaded Text Data
Loaded Relation Data
Loaded Profile Data
Loaded Image Data


In [66]:
#code for loading pickles
pickle_in = open("combined_df_merged.pkl","rb")
data_merged = pickle.load(pickle_in)


In [60]:
#various data loading and preprocessing 



data_emotional = data[0]
data_text = data_text.rename(columns={0:'userId'})
#data_merged.iloc[:,2] = data_merged.iloc[:,2].apply(lambda x: 0 if x < 25 else (1 if x < 35 else (2 if x < 50 else 3)))
data_text.iloc[:,2] = data_text.iloc[:,2].apply(lambda x: 0 if x < 25 else (1 if x < 35 else (2 if x < 50 else 3)))

#data_text = data_text.sort_values(by=[0])
#data_merged = data_merged.sort_values(by=[0])
#data_emotional = data_emotional.sort_values(by=['userId'])
combined_data = pd.merge(data_text, data_emo, on='userId')


In [ ]:
#age on combined LIWC/NRC
emo_age = GradientBoostingClassifier(n_estimators=400,max_depth=5,learning_rate=0.01)
split = 5
kf = KFold(n_splits=split)
result = []
data = combined_data
for train_index, test_index in kf.split(data):
    c = 0
    train = data.iloc[train_index,8:]
    test = data.iloc[test_index,8:]
    train_labels = data.iloc[train_index,2]
    test_labels = data.iloc[test_index,2]
    emo_age.fit(train, train_labels)
    t = emo_age.predict(test)
    for i in range(len(test_labels)):
        if t[i] == test_labels.iloc[i]:
            c+=1
    result.append(c/len(test_labels))
#print(result)
#print(sum(result)/len(result))


In [ ]:
#gender on facial 
emo_gen = GradientBoostingClassifier(n_estimators=400,max_depth=5,learning_rate=0.01)
split = 5
kf = KFold(n_splits=split)
result = []
data = data_merged
for train_index, test_index in kf.split(data):
    c = 0
    train = data.iloc[train_index,3:]
    test = data.iloc[test_index,3:]
    train_labels = data.iloc[train_index,1]
    test_labels = data.iloc[test_index,1]
    emo_gen.fit(train, train_labels)
    t = emo_gen.predict(test)
    for i in range(len(test_labels)):
        if t[i] == test_labels.iloc[i]:
            c+=1
    result.append(c/len(test_labels))
#print(result)
#print(sum(result)/len(result))

In [8]:
pickle.dump(emo_gen, open( "gender_model.pkl", "wb" ) )

In [224]:
#gradient boost can be replaced with any of the necessaey functions
#the function can also be passed in as an argument, but this wasn't done here
def run_functions(j,k):
    r = []
    emo_age = GradientBoostingClassifier(n_estimators=400,max_depth=5,learning_rate=0.01)
    split = 5
    kf = KFold(n_splits=split)
    pca = PCA(n_components=2)
    count = 0
    data = data_text
    for train_index, test_index in kf.split(data):
        train = data.iloc[train_index,np.r_[j,k]]
        test = data.iloc[test_index,np.r_[j,k]]
        train_labels = data.iloc[train_index,2]
        test_labels = data.iloc[test_index,2]
        log.fit(train, train_labels)
        t = log.predict(test)
        c = 0
        for i in range(len(test_labels)):
            if t[i] == test_labels.iloc[i]:
                c+=1
        result.append(c/len(test_labels))
    r.append(sum(result)/len(result))
    print("{} and {} index is {}".format(k,j,sum(result)/len(result)))    
    return r



#runs a loop over a permutation of n different features 
results = []
indices = []
for i in range(8,len(data_text.columns)):
    for j in range(8, len(data_text.columns)):
        if i != j:
            results.append(run_functions(i,j))
            indices.append((i,j))
    print("Finished one outer loop feature")



In [ ]:
#original code to combine dataframes - deprecated when an easier method was developped

data_text['userId'] = data_text['userId'].astype('str')
data_profile['userid'] = data_profile['userid'].astype('str')
data_profile.set_index('userid')
data_facial.set_index('userId')

data_user = pd.DataFrame()
data_tex = pd.DataFrame()
d = []
el = []
for i in range(len(data_profile)):
    for j in range(len(data_text)):
        user = []
        if data_profile.loc[i,'userid'] == data_text.loc[j,'userId']:
            user.append(data_profile.loc[i,'userid'])
            user.append(data_profile.loc[i,'gender'])
            user.append(data_profile.loc[i,'age'])
            user.append(data_profile.loc[i,'ope'])
            user.append(data_profile.loc[i,'con'])
            user.append(data_profile.loc[i,'ext'])
            user.append(data_profile.loc[i,'agr'])
            user.append(data_profile.loc[i,'neu'])
            el.append(data_text.iloc[j,1:])
            if len(user) != 0:
                d.append(user)

d1 = pd.DataFrame(d)
d2 = pd.DataFrame(el)
d1.reset_index(drop=True, inplace=True)
d2.reset_index(drop=True, inplace=True)
frames = [d1, d2]
d3 = pd.concat(frames, axis=1)
d3.head()
d3.to_pickle("combined_df_text.pkl")            



In [ ]:
#we tried PCA and kmeans initially but it didn;t work so well - code left for interest only
class pca_kmeans():
    def __init__(self,components, splits, data):
        self.results = []
        self.comp = components
        self.splits = splits
        self.data = data
        
    def run(self):
        for i in range(self.comp):
            self.kfold(i)
            print("Component {} out of {} done".format(i+1, self.comp))
            print(self.results[i])
        return self.results
        
    def kfold(self, num_comp):
        kf = KFold(n_splits=self.splits)
        temp = []
        for train_index, test_index in kf.split(self.data):
            self.train = data.iloc[train_index,:]
            self.test = data.iloc[test_index,:]
            self.pca(num_comp)
            t = sum(self.kmeans())/test_index.shape[0]
            if t < 0.5:
                t = 1 -t
            temp.append(t)
        mean = sum(temp)/len(temp)
        self.results.append(mean)
        
    def pca(self, num_comp):
        pca = PCA(n_components = num_comp+1)
        pca.fit(self.train)
        self.f = pca.transform(self.train)
        self.features = pca.transform(self.test)
        
    def kmeans(self):
        kmeans = KMeans(init='k-means++', n_clusters=2, n_init=25).fit(self.f)
        return kmeans.predict(self.features)
        
#result = a.run()
#result_n = b.run()
#print("The result for a is ",sum(result)/len(result))
#print("The result for b is ", sum(result_n)/len(result_n))